# LSTM XOR
We want to implement an LSTM-RNN in TF to predict the XOR operation over a sequence of fixed length (for now) of bits.
First we need to generate these sequences and to compute the XOR itself.

Remarks:
- The XOR operation has a non-linear decision hyperplane

In [1]:
import tensorflow as tf
import numpy as np

/Users/nicom/anaconda/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
